## Pick Adverse Events from MIMIC3

In [1]:
import pandas as pd
import os
from os.path import join
import numpy as np
import matplotlib.pyplot as plt

from tools import *

In [2]:
cluster_prefix = "/data/liu/mimic3/CLUSTER"

In [4]:
dfs = []
dfs_control = []
for i in range(4):
    dfs = dfs + [read_data(join(cluster_prefix,"kmeans_%d_potential_disease"%i),dtype={"ICD9_CODE":str})]
    dfs_control = dfs_control + \
    [read_data(join(cluster_prefix,"kmeans_%d_control_potential_disease"%i),dtype={"ICD9_CODE":str})]

In [5]:
dfs[0].head()

,ICD9_CODE,HADM_ID
0,4019,4733
1,42731,2735
2,5849,2650
3,2724,2430
4,4280,2241


In [6]:
dfs_control[0].head()

,ICD9_CODE,HADM_ID
0,4019,6490
1,41401,5393
2,42731,4032
3,4280,3866
4,2724,3585


In [7]:
## Side effects
sideffect_prefix = '/data/liu/adverse_events'
ade_df = read_data(join(sideffect_prefix, 'ndc_icd9_side_effects'), dtype={'NDC':str, 'ICD_CODE':str})
ade_df.head()

/home/liu/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


,NDC,drugbank_id,UMLS_CUI,ICD_CODE,SNOMED_CID,ICD9_CODE
0,00003029305,DB00620,C0000731,789.36,60728008,78936
1,00003049420,DB00620,C0000731,789.36,60728008,78936
2,00009001104,DB00620,C0000731,789.36,60728008,78936
3,00009001201,DB00620,C0000731,789.36,60728008,78936
4,00009003101,DB00620,C0000731,789.36,60728008,78936


## Simply defined potential adverse events, frequency in treatment group (taking the drug) > frequency in control group (not taking the drug)

In [12]:
# compare_c1= left_join(dfs[0],dfs_control[0],"ICD9_CODE")
# potential_ade_c1 = compare_c1[compare_c1['HADM_ID_x']>compare_c1['HADM_ID_y']]
potential_ade_c1.head()

,ICD9_CODE,HADM_ID_x,HADM_ID_y
2,5849,2650,1632.0
7,5990,1825,1235.0
8,51881,1789,815.0
10,2851,1531,1262.0
11,2762,1361,652.0


In [18]:
non_potential_ade_c1 = compare_c1[compare_c1['HADM_ID_x'] <= compare_c1['HADM_ID_y']]

In [16]:
# drug_ndc = '00338004904'
true_ade = ade_df[ade_df['NDC']==drug_ndc][['ICD9_CODE']]
true_ade['match']=1
true_ade.head()

,ICD9_CODE,match
1696,78936,1
4090,6829,1
4091,6828,1
9124,9950,1
31876,7993,1


In [17]:
left_join(potential_ade_c1, true_ade,"ICD9_CODE")

,ICD9_CODE,HADM_ID_x,HADM_ID_y,match
0,5849,2650,1632.0,1.0
1,5990,1825,1235.0,NaN
2,51881,1789,815.0,NaN
3,2851,1531,1262.0,NaN
4,2762,1361,652.0,1.0
5,99592,1357,425.0,NaN
6,486,1187,831.0,NaN
7,0389,1128,455.0,1.0
8,2761,1076,514.0,NaN
9,40390,1028,1000.0,NaN


In [19]:
left_join(non_potential_ade_c1, true_ade,"ICD9_CODE")

,ICD9_CODE,HADM_ID_x,HADM_ID_y,match
0,4019,4733,6490.0,NaN
1,42731,2735,4032.0,NaN
2,2724,2430,3585.0,NaN
3,4280,2241,3866.0,NaN
4,25000,1967,2730.0,NaN
5,41401,1850,5393.0,NaN
6,53081,1597,2094.0,NaN
7,2859,1356,1489.0,NaN
8,2449,1278,1340.0,NaN
9,V5861,1110,1229.0,NaN
